## General Adversial Network (GAN)

Generative Adversarial Networks (GANs) are a framework for training networks optimized for generating new realistic samples from a particular representation. In its simplest form, the training process involves two networks. One network, called the `Generator`, generates new data instances, trying to fool the other network, the `Discriminator`, that classifies images as real or fake. 
<img src="https://camo.githubusercontent.com/22afb5278428d6c4c785d1e62639b6dcafc73c7a59b64e86f6d4f71ed54df094/68747470733a2f2f692e696d6775722e636f6d2f4c7765614431732e706e67"/>

There are broadly 3 categories of `GANs`.
1. <b>Unsupervised GANs</b>: The Generator network takes random noise as input and produces a photo-realistic image that appears very similar to images that appears in the training dataset. Example `DC-GAN`, `pg-GAN`
2. <b>Style-Transfer GANs</b>: Translate images from one domain to another. Examples like `CycleGAN` and `Pix2Pix`
3. <b>Conditional GANs</b>: Jointly learn on features along with images to generate images conditioned on those features. Example like `Conditional GAN`, `AC-GAN` and `BigGAN`.

### Part 1: BigGAN

In [17]:
# Install imageio-ffmpeg
!pip install imageio-ffmpeg

In [18]:
import io
import os
import numpy as np

# Deep Learning
from scipy.stats import truncnorm
import tensorflow as tf
import tensorflow_hub as hub

# Visualization
from IPython.core.display import HTML
import imageio
import base64

# Checks that TensorFlow GPU is Enabled
tf.test.gpu_device_name()   

### Load  BigGAN Generator Module from TFHub 

In [19]:
# Remove EagerExecution in TF2.x for preventing this Error
# RuntimeError: Exporting/importing meta graphs is not supported when eager execution is enabled. No graph exists when eager execution is enabled.
tf.compat.v1.disable_eager_execution()

module_path = 'https://tfhub.dev/deepmind/biggan-512/1' # 512x512 BigGAN

tf.compat.v1.reset_default_graph()
print('Loading BigGAN module from: ', module_path)
module = hub.Module(module_path)
inputs = {k: tf.compat.v1.placeholder(v.dtype, v.get_shape().as_list(), k) for k, v in module.get_input_info_dict().items()}
output = module(inputs)

In [20]:
outputs

### Functions for Sampling and Interpolating the Generator

In [21]:
input_z = inputs['z']
input_y = inputs['y']
input_trunc = inputs['truncation']

dim_z = input_z.shape.as_list()[1]
vocab_size = input_y.shape.as_list()[1]

# Sample Truncated Normal distribution based on Seed and Truncation Parameter
def truncated_z_sample(truncation=1., seed=None):
    state = None if seed is None else np.random.RandomState(seed)
    values = truncnorm.rvs(-2, 2, size=(1, dim_z), random_state=state)
    return truncation * values

# Convert `index` value to a vector of all zeros except for a 1 at `index`
def one_hot(index, vocab_size=vocab_size):
    index = np.asarray(index)
    if len(index.shape) == 0: # when it's a scale convert to a vector of size 1
        index = np.asarray([index])
    assert len(index.shape) == 1
    num = index.shape[0]
    output = np.zeros((num, vocab_size), dtype=np.float32)
    output[np.arange(num), index] = 1
    return output

def one_hot_if_needed(label, vocab_size=vocab_size):
    label = np.asarray(label)
    if len(label.shape) <= 1:
        label = one_hot(label, vocab_size)
    assert len(label.shape) == 2
    return label

# Using vectors of noise seeds and category labels, generate images
def sample(sess, noise, label, truncation=1., batch_size=8, vocab_size=vocab_size):
    noise = np.asarray(noise)
    label = np.asarray(label)
    num = noise.shape[0]
    if len(label.shape) == 0:
        label = np.asarray([label] * num)
    if label.shape[0] != num:
        raise ValueError('Got # noise samples ({}) != # label samples ({})'
                         .format(noise.shape[0], label.shape[0]))
    label = one_hot_if_needed(label, vocab_size)
    ims = []
    for batch_start in range(0, num, batch_size):
        s = slice(batch_start, min(num, batch_start + batch_size))
        feed_dict = {input_z: noise[s], input_y: label[s], input_trunc: truncation}
        ims.append(sess.run(output, feed_dict=feed_dict))
    ims = np.concatenate(ims, axis=0)
    assert ims.shape[0] == num
    ims = np.clip(((ims + 1) / 2.0) * 256, 0, 255)
    ims = np.uint8(ims)
    return ims

def interpolate(a, b, num_interps):
    alphas = np.linspace(0, 1, num_interps)
    assert a.shape == b.shape, 'A and B must have the same shape to interpolate.'
    return np.array([(1-x)*a + x*b for x in alphas])

def interpolate_and_shape(a, b, steps):
    interps = interpolate(a, b, steps)
    return (interps.transpose(1, 0, *range(2, len(interps.shape))).reshape(steps, -1))

### Create a TensorFlow Session and initialize Variables

In [22]:
initializer = tf.compat.v1.global_variables_initializer()
sess = tf.compat.v1.Session()
sess.run(initializer)

### Create Video of Interpolated BigGAN Generator Samples

In [23]:
# Category options: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a
category = 947 # mushroom

# Important parameter that controls how much variation there is
truncation = 0.2 # reasonable range: [0.02, 1]

seed_count = 10
clip_secs = 36

seed_step = int(100 / seed_count)
interp_frames = int(clip_secs * 30 / seed_count)  # interpolation frames

cat1 = category
cat2 = category
all_imgs = []

for i in range(seed_count):
    seed1 = i * seed_step # good range for seed is [0, 100]
    seed2 = ((i+1) % seed_count) * seed_step
    
    z1, z2 = [truncated_z_sample(truncation, seed) for seed in [seed1, seed2]]
    y1, y2 = [one_hot([category]) for category in [cat1, cat2]]

    z_interp = interpolate_and_shape(z1, z2, interp_frames)
    y_interp = interpolate_and_shape(y1, y2, interp_frames)

    imgs = sample(sess, z_interp, y_interp, truncation=truncation)
    
    all_imgs.extend(imgs[:-1])

# Save the video for displaying in the next cell, this is way more space efficient than the gif animation
imageio.mimsave('gan.mp4', all_imgs, fps=30)

In [24]:
%%HTML
<video autoplay loop>
  <source src="gan.mp4" type="video/mp4">
</video>